# Analyzing Edit Activity of Zurich

In [5]:
from wikidata.client import Client
import requests
import json
from io import StringIO

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Zurich

In [2]:
zurich = 'Q72'
canton_zurich = 'Q11943'

client = Client()
print(client.get(zurich, load=True))
print(client.get(canton_zurich, load=True))

<wikidata.entity.Entity Q72 'Zürich'>
<wikidata.entity.Entity Q11943 'canton of Zürich'>


## Get the Complete Edit Activity For an Item

In [17]:
def edit_hist(item):
    fields = ['timestamp', 'user', 'userid', 'comment', 'size']
    field_str = '%7C'.join(fields)
    limit = '500'
    query_template = 'https://www.wikidata.org/w/api.php?action=query&format=json&prop=revisions&titles={item}&rvprop={fields}&rvslots=main&rvlimit={limit}'
    query = query_template.format(
        item=item, fields=field_str, limit=limit)

    curr_hist_dict = requests.get(query).json()

    res_str = 'UserID,User,Timestamp,Size,Comment\n'

    while True:
        for page in curr_hist_dict['query']['pages'].values():
            for revision in page['revisions']:
                res_str += str(revision['userid']) + ','
                res_str += revision['user'] + ','
                res_str += revision['timestamp'] + ','
                res_str += str(revision['size']) + ','
                res_str += '"' + revision['comment'] + '"'+ '\n'

        try:
            continue_field = curr_hist_dict['continue']['rvcontinue']
            new_query = query + '&rvcontinue=' + continue_field
            curr_hist_dict = requests.get(new_query).json()
        except KeyError:
            break
            
    return pd.read_csv(StringIO(res_str))

## Analysis

In [21]:
edit_hist(zurich)

,UserID,User,Timestamp,Size,Comment
0,54747,Civvì,2018-08-28T12:47:30Z,540538,/* wbcreateclaim-create:1| */ [[Property:P1435...
1,728752,SM7,2018-08-26T18:26:02Z,540108,"/* wbsetaliases-add:4|bho */ ज्युरिख, जूरिख, ज..."
2,728752,SM7,2018-08-26T18:25:58Z,539788,/* wbsetdescription-add:1|bho */ स्विट्जरलैंड ...
3,728752,SM7,2018-08-26T18:25:55Z,539571,/* wbsetlabel-add:1|bho */ ज्यूरिख
4,1110636,Arch2all,2018-08-06T10:02:32Z,539489,/* wbsetreference-add:2| */ [[Property:P17]]: ...
5,2831262,MsynBot,2018-08-06T02:00:12Z,538420,/* wbeditentity-update:0| */ update reference ...
6,1110636,Arch2all,2018-08-05T10:49:59Z,538179,/* wbsetreference-add:2| */ [[Property:P5573]]...
7,1110636,Arch2all,2018-08-05T10:49:56Z,537069,/* wbcreateclaim-create:1| */ [[Property:P5573...
8,2831262,MsynBot,2018-07-27T15:01:51Z,536730,/* wbeditentity-update:0| */ update reference(...
9,5462,Vlsergey,2018-07-25T04:57:45Z,537044,/* wbeditentity-update:0| */ via [[:w:ru:ВП:WE...
